In [13]:
import os

from sqlalchemy import and_, or_
import pandas as pd
pd.set_option("display.max_rows", None)

from gresq.config import Config
from gresq.database.v1_1_0 import dal as db1     # prod dal
from gresq.database import dal as db2            # test dal
from gresq.database.v1_1_0 import Base as Base1  # prod base
from gresq.database import Base as Base2         # test base

from gresq.database.v1_1_0.models import Sample,Recipe,Author


In [14]:
# Setup old schema (production db) as db1 using read only
config_prefix = "PROD_DATABASE"
config_suffix = "_READ"
conf = Config(prefix=config_prefix, suffix=config_suffix, debug=True, try_secrets=False)
db1.init_db(conf, privileges={"read": True, "write": False, "validate": False})
# Testing Configuration and dal initialization
config_prefix = "TEST_DATABASE"
config_suffix = "_ADMIN" 
conf = Config(prefix=config_prefix, suffix=config_suffix, debug=True, try_secrets=False)
db2.init_db(conf, privileges={"read": True, "write": True, "validate": True})

# 0) Stand up fresh empty db2
ses2 = db2.Session()
ses2.close()
Base2.metadata.drop_all(bind=db2.engine)
Base2.metadata.create_all(bind=db2.engine)

In [15]:
# Get all of the samples with their recipe data
ses1 = db1.Session()
pd.read_sql(
    ses1.query(Sample,Recipe).filter(Sample.id==Recipe.sample_id)
    .statement,
    ses1.bind,
)
# For each row:
# 1) Create Expirment row, use ll 1:1 mapped columns
# 1) Get author info from sample.recipe
# 2) create author row (if not exists)
# 3) Add author id to 

,id,software_name,software_version,primary_sem_file_id,nanohub_userid,experiment_date,material_name,validated,id,sample_id,thickness,diameter,length,catalyst,tube_diameter,cross_sectional_area,tube_length,base_pressure,dewpoint,sample_surface_area


In [16]:
ses1 = db1.Session()
unique_substrate_query = ses1.query(Recipe.catalyst, Recipe.thickness, Recipe.diameter, Recipe.length, Recipe.sample_surface_area).distinct().filter(
        or_(
            Recipe.catalyst != None,
            Recipe.thickness != None,
            Recipe.diameter != None,
            Recipe.sample_surface_area != None,
            Recipe.length != None,
        )
    )

pd.read_sql(
    unique_substrate_query.statement,
    ses1.bind,
)

,catalyst,thickness,diameter,length,sample_surface_area


In [17]:
# Test query for furnace data

ses1 = db1.Session()
unique_furnace_query = ses1.query(Recipe.tube_diameter, Recipe.cross_sectional_area, Recipe.tube_length).distinct().filter(
        or_(
            Recipe.tube_diameter != None,
            Recipe.cross_sectional_area != None,
            Recipe.tube_length != None,
        )
    )
pd.read_sql(
    unique_furnace_query.statement,
    ses1.bind,
)

,tube_diameter,cross_sectional_area,tube_length


In [18]:
# Read all unique authors from old schema
from gresq.database.v1_1_0.models import Author

ses1 = db1.Session()
#unique_authors_query = ses1.query(
#        Author.sample_id,Author.first_name, Author.last_name, Author.institution
#    ).filter(Author.sample_id==Sample.id).group_by(Author.sample_id,Author.first_name, Author.last_name)
unique_authors_query = ses1.query(
        Author.first_name, Author.last_name, Author.institution
    ).distinct()
pd.read_sql(
    unique_authors_query.statement,
    ses1.bind,
)

,first_name,last_name,institution


In [20]:
ses1 = db1.Session
authors = ses1.query(
    Author.sample_id, Author.first_name, Author.last_name, Author.institution
).group_by(Author.first_name, Author.last_name)
ses1.query(Sample.id,Sample.nanohub_userid).filter(Sample.id.in_(authors))

In [21]:
ses1 = db1.Session()
# authors = ses1.query(Author.first_name, Author.last_name, Author.institution).distinct()
samples = ses1.query(Sample,Recipe).filter(Sample.id==Recipe.sample_id)
for s in samples:
    print(f"name: {s.Sample.material_name}")
    print(f"{s.Recipe.base_pressure,s.Recipe.catalyst,s.Recipe.thickness,s.Recipe.diameter,s.Recipe.length,s.Recipe.sample_surface_area}")

In [12]:
######################################################################################
# Migration script
# ToDo:
# - TESTING!!!
# - nanohub_userid
# - environment_conditions
# - sem_files + analysis
# - raman_files + analysis
# - software versions
######################################################################################

from gresq.database.v1_1_0.models import Author
from gresq.database.models import Author as Author2
from gresq.database.models import Furnace as Furnace2
from gresq.database.models import Experiment as Experiment2
from gresq.database.models import Recipe as Recipe2
from gresq.database.models import PreparationStep as PreparationStep2
from gresq.database.models import Substrate as Substrate2

commit = True
limit = 200

from math import isclose

# 0) Stand up fresh empty db2
print("Clearing destination database...")
ses2 = db2.Session()
ses2.close()
Base2.metadata.drop_all(bind=db2.engine)
Base2.metadata.create_all(bind=db2.engine)
print("done.")




print("Adding furnace rows...")
furnaces = unique_furnace_query
new_furnaces = [
    Furnace2(
        tube_diameter =  f.tube_diameter,
        cross_sectional_area = f.cross_sectional_area,
        tube_length = f.tube_length
    ) for f in furnaces
]
if commit:
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_furnaces)
print("done.")
       

print("Adding substrate rows...")
substrates = unique_substrate_query
new_substrates = [
    Substrate2(
        catalyst = s.catalyst,
        thickness = s.thickness,
        diameter = s.diameter,
        surface_area = s.sample_surface_area,
        length = s.length
    ) for s in substrates
]
if commit:
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_substrates)
print("done.")

print("Adding author rows...")
authors = unique_authors_query
new_authors = [
    Author2(
        first_name = a.first_name,
        last_name = a.last_name,
        institution = a.institution
    ) for a in authors
]

# Submit type of call/ insert/update
if commit:
    # call factory function gives live session from class 
    with db2.session_scope(autocommit=True) as ses2:
        ses2.add_all(new_authors)
print("done.")

print("Getting old sample data...")
ses1 = db1.Session()
samples = ses1.query(Sample,Recipe).filter(Sample.id==Recipe.sample_id).limit(limit)
oauthors = ses1.query(Author).all()
print("done.")

#####################################################################################
# At this point unique rows for furnace, substrate and Authors have been created
# Now, Loop back through the samples and match rows in furnace, substrate and authors
# Build a map from Experiment id  -> furnace_id, etc. for each sample/experiment id
# Note this assumes that the experiment id will map to the (old) sample id.
######################################################################################
ses3 = db2.Session()
new_furnaces = ses3.query(Furnace2).all()
new_substrates = ses3.query(Substrate2).all()
new_authors = ses3.query(Author2).all()
    
furnacemap = {}
substratemap = {}
authormap = {}
tol = 0.01
print ("Matching parent tables...")
for s in samples:
    authormap[s.Sample.id]=[]
    #print(f"name: {s.Sample.material_name}")
    #print(f"{s.Recipe.base_pressure,s.Recipe.catalyst}")
    
    #################################################################################
    # Match to a row in the furnace table
    #################################################################################
    for f in new_furnaces:
        #print(f"id: {f.id}")
        match = match1 = match2 = match3 = False
        null = null1 = null2 = null3 = False
        if f.tube_diameter:
            if s.Sample.recipe.tube_diameter:
                if isclose(f.tube_diameter,s.Sample.recipe.tube_diameter,rel_tol=0.001,abs_tol=0.001):
                    match1 = True
        else:
            if not s.Sample.recipe.tube_diameter:
                match1 = True
                null1 = True
        if f.tube_length:
            if s.Sample.recipe.tube_length:
                if isclose(f.tube_length,s.Sample.recipe.tube_length,rel_tol=0.001,abs_tol=0.001):
                    match2 = True
        else:
            if not s.Sample.recipe.tube_length:
                match2 = True
                null2 = True
        if f.cross_sectional_area:
            if s.Sample.recipe.cross_sectional_area:
                if isclose(f.cross_sectional_area,s.Sample.recipe.cross_sectional_area,rel_tol=0.001,abs_tol=0.001):
                    match3 = True
        else:
            if not s.Sample.recipe.cross_sectional_area:
                match3 = True
                null3 = True
        # If there are any non null matches    
        match = match1 and match2 and match3
        if match:
            furnacemap[s.Sample.id] = f.id
            #print(f"match: {s.Sample.id}: {f.id}")
            break
        else:
            furnacemap[s.Sample.id] = None
            
        
    #################################################################################
    # Match to row in Substrate table
    #################################################################################
    for sb in new_substrates:
        match = match1 = match2 = match3 = match4 = match5 = False
        if sb.catalyst:
            if sb.catalyst == s.Sample.recipe.catalyst:
                match1 = True
        else:
            if not s.Sample.recipe.catalyst:
                match1 = True
        if sb.thickness:
            if s.Sample.recipe.thickness:
                if isclose(sb.thickness,s.Sample.recipe.thickness,rel_tol=tol,abs_tol=tol):
                    match2 = True
        else:
            if not s.Sample.recipe.thickness:
                match2 = True
        if sb.diameter:
            if s.Sample.recipe.diameter:
                if isclose(sb.diameter,s.Sample.recipe.diameter,rel_tol=tol,abs_tol=tol):
                    match3 = True
        else:
            if not s.Sample.recipe.diameter:
                match3 = True
        if sb.length:
            if s.Sample.recipe.length:
                if isclose(sb.length,s.Sample.recipe.length,rel_tol=tol,abs_tol=tol):
                    match4 = True
        else:
            if not s.Sample.recipe.length:
                match4 = True
        if sb.surface_area:
            if s.Sample.recipe.sample_surface_area:
                if isclose(sb.surface_area,s.Sample.recipe.sample_surface_area,rel_tol=tol,abs_tol=tol):
                    match5 = True
        else:
            if not s.Sample.recipe.sample_surface_area:
                match5 = True
        match = match1 and match2 and match3 and match4 and match5
        if match:
            substratemap[s.Sample.id] = sb.id
            break
        else:
            substratemap[s.Sample.id] = None
    #################################################################################
    # Match to row(s) in Author table
    #################################################################################
    for oa in oauthors:
        if not oa.sample_id == s.Sample.id:
            continue
        for na in new_authors:
            match = match1 = match2 = False
            if oa.first_name == na.first_name:
                match1 = True
            if oa.last_name == na.last_name:
                match2 = True
            match = match1 and match2
            if match:
                authormap[s.Sample.id].append(na)
                break
        
        
    

#Print out the maps
print (f"fmap: {furnacemap}")
print (f"sbmap: {substratemap}")
print(f"amap: {authormap}")
print ("Done.")        

#####################################################################################
#####################################################################################

print("Adding Experiment rows...")
with db2.session_scope(autocommit=True) as ses2:
    for s in samples:
        #print(f"{s.Sample.id}")
        try:
            carbon_source = s.Recipe.carbon_source
        except IndexError:
            carbon_source = None
        

            
        # Add the new experiment based on the old sample
        # Create a recipe for each old sample and add all of its prep steps
        newexp = Experiment2(
                id=s.Sample.id,
                furnace_id = furnacemap[s.Sample.id],
                substrate_id = substratemap[s.Sample.id],
                authors = authormap[s.Sample.id],
                material_name=s.Sample.material_name,
                experiment_date=s.Sample.experiment_date,
                recipe = Recipe2(
                    carbon_source=carbon_source,
                    base_pressure=s.Recipe.base_pressure,
                    preparation_steps = [
                        PreparationStep2(
                            name=step.name,
                            duration=step.duration
                        ) for step in s.Recipe.preparation_steps
                    ]
                    ),
                
                )
        
        ses2.add(newexp)
print("done.")
      


ModuleNotFoundError: No module named 'gresq.database.models.raman_spectrum'

In [38]:
%env

{'USER': 'root',
 'VSCODE_WSL_EXT_LOCATION': '/mnt/c/Users/byoon/.vscode/extensions/ms-vscode-remote.remote-wsl-0.56.2',
 'SHLVL': '0',
 'HOME': '/root',
 'WSL_DISTRO_NAME': 'Ubuntu',
 'LOGNAME': 'root',
 'NAME': 'DESKTOP-G15VBC3',
 '_': '/root/.vscode-server/bin/cfa2e218100323074ac1948c885448fdf4de2a7f/node',
 'TERM': 'xterm-color',
 'PATH': '/root/.vscode-server/bin/cfa2e218100323074ac1948c885448fdf4de2a7f/bin:/root/.nvm/versions/node/v14.16.0/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/mnt/c/windows/system32:/mnt/c/windows:/mnt/c/windows/System32/Wbem:/mnt/c/windows/System32/WindowsPowerShell/v1.0/:/mnt/c/windows/System32/OpenSSH/:/mnt/c/Program Files/Git/cmd:/mnt/c/Program Files/Microsoft SQL Server/130/Tools/Binn/:/mnt/c/Program Files (x86)/Windows Kits/8.1/Windows Performance Toolkit/:/mnt/c/Program Files/MySQL/MySQL Shell 8.0/bin/:/mnt/c/Users/byoon/AppData/Local/Microsoft/WindowsApps:/mnt/c/Users/byoon/AppData/Local/Programs/Mic